## 数据结构和算法
### 1 将序列分解为单独的变量
#### 1-1 问题
我们有一个包含N个元素的元组或序列，现在想将它分解为N个单独的变量。
#### 1-2 解决方案
任何序列(或可迭代的对象)都可以通过一个简单的复制操作来分解为单独的变量。唯一的要求就是变量的总数和结构要与序列相吻合。例如:

In [3]:
p = (4,5)
x, y = p
x

4

In [4]:
y

5

In [8]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, date = data
name

'ACME'

In [10]:
date

(2012, 12, 21)

In [11]:
name, shares, price, (year, mon, day) = data
name

'ACME'

In [12]:
year

2012

如果元素的数量不匹配，将得到一个错误的提示。例如:

In [13]:
p = (4,5)
x, y, z = p
x

ValueError: not enough values to unpack (expected 3, got 2)

#### 1-3 讨论

实际上不仅仅只是元组或列表，只要对象恰好是可迭代的，那么就可以执行分解操作。这包括字符串，文件，迭代器以及生成器。比如:

In [14]:
s = 'hello'
a, b, c, d, e = s
a

'h'

In [15]:
e

'o'

当做分解操作时，有时候可能想丢弃某些特定的值。python并没有提供特殊的语法来实现这一点，但是通常可以选一个用不到的变量名，以此来作为丢弃的值的名称。例如:

In [16]:
data = ['ACME', 50, 90.1, (2012, 12, 21)]
_, shares, price, _ = data
shares

50

In [17]:
price

90.1

### 2 从任意长度的可迭代对象中分解元素
#### 2-1 问题
需要从某个可迭代对象中分解出N个元素，但是这个可迭代对象的长度可能超过N，这会导致出现“分解的值过多”的异常。
#### 2-2 解决方案
Python的“*表达式”可以用来解决这个问题。例如，假设开设了一门课程，并决定在期末作业成绩中去掉第一个和最后一个，只对中间剩下的成绩做平均统计。如果只有4个成绩，也许可以将4个都分解出来，但是如果有24个呢？“*表达式”使这一切变得简单:

In [18]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

另一个用例是假设有一些用户记录，记录由姓名和电子邮件地址组成，后面跟着任意数量的电话号码。则可以像这样分解记录:

In [20]:
user_record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = user_record
name

'Dave'

In [21]:
email

'dave@example.com'

In [24]:
phone_numbers

['773-555-1212', '847-555-1212']

由*修饰的变量也可以位于列表的第一个位置。例如，比方说用一系列的值，来代表公司过去8个季度的销售额。如果相对最近一个季度的销售额同前7个季度的平均值作比较，可以这么做:

In [29]:
*trailing, current = [10,8,8,8,8,8,7]
trailing

[10, 8, 8, 8, 8, 8]

In [30]:
current

7

#### 2-3 讨论
对于分解未知或任意长度的可迭代对象，这种扩展的分解操作可谓是量身定制的工具。通常，这类可迭代对象中会有一些已知的组件或模式(例如，元素1之后的所有内容都是电话号码)。
*式语法在迭代一个变长的元组序列时尤其有用。例如，假设有一个带标记的元组序列:

In [31]:
records = [
    ('foo', 1, 2),
    ('bar','hello'),
    ('foo', 3, 4),
    ]

def do_foo(x,y):
    print('foo',x,y)
def do_bar(s):
    print('bar',s)
    
for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


当和某些特定的字符串处理操作箱结合，比如做拆分（splitting）操作时，这种*式的语法所支持的分解操作也非常有用。例如:

In [32]:
line = 'nobody:*:-2:-2:Unpeivileged User:/var/empty:/user/bin/false'
uname, *fields, homedir, sh = line.split(':')
uname

'nobody'

In [33]:
homedir

'/var/empty'

In [34]:
sh

'/user/bin/false'

有时候可能想分解出某些值然后丢弃它们。在分解的时候，不能只是指定一个单独的*，但是可以使用几个常用的来表示待丢弃的变量名，比如_或者ign(ignored)。例如:

In [37]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
name

'ACME'

In [38]:
year

2012

*分解操作和各种函数式语言中的列表处理功能有着一定的相似性。例如，如果有一个列表，可以像下面这样轻松将其分解为头部和尾部:

In [39]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items
head

1

In [40]:
tail

[10, 7, 4, 5, 9]

在编写执行这类拆分功能的函数时，人们可以假设这是为了实现某种精巧的递归算法。例如:

In [43]:
def sum(items):
    head, *tail = items
    return head + sum(tail) if tail else head

sum(items)

36

### 3保存最后N个元素
#### 3-1 问题
我们希望在迭代或是其他形式的处理过程中对最后几项记录做一个有限的历史记录。
#### 3-2 解决方案
保存有限的历史记录可算是collections.deque的完美应用场景了。例如，下面的代码对一系列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本。

In [51]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines
        previous_lines.append(line)
        
# Example use on a file
if __name__ == '__main__':
    with open('somefile.txt') as f:
        for line, previous in search(f, 'python', 5):
            for pline in previous:
                print(pline, end='')
            print(line, end='')
            print('-'*20)

python
--------------------
python
eeepython
--------------------
python
eeepython
ddpython
--------------------
python
eeepython
ddpython
pffython
dddpython
--------------------
ddpython
pffython
dddpython
ffptyfh
dfh
python1
--------------------
pffython
dddpython
ffptyfh
dfh
python1
python2--------------------


#### 3-3 讨论
如同上面的代码片段中所做的一样，当编写搜索某项记录的代码时，通常会用到含有yield关键字的生成器函数。这将处理搜索过程的代码和使用搜索结果的代码成功解耦开来。
deque(maxlen=N)创建了一个固定长度的队列。当有新纪录加入而队列已满时会自动移除最老的那条记录。例如:

In [55]:
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [56]:
q.append(4)
q

deque([2, 3, 4])

In [57]:
q.append(5)
q

deque([3, 4, 5])

更普遍的是，当需要一个简单的队列结构时，deque可助你一臂之力。如果不指定队列的大小，也就得到无界限的队列，可以再两端执行添加和弹出操作，例如:

In [58]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [59]:
q.appendleft(4)
q

deque([4, 1, 2, 3])

In [61]:
q.pop()

3

In [62]:
q

deque([4, 1, 2])

In [63]:
q.popleft()

4

从队列两端添加或弹出元素的复杂度都是O(1)。这和列表不同，当从列表的头部插入或移除元素时，列表的复杂度为O(n)。

### 4 找到最大或最小的N个元素
#### 4-1 问题
我们想在某个集合中找出最大或最小的N个元素。
#### 4-2 解决方案
heapq模块中有两个函数,nlargest()和nsmallest()，它们正是我们所需要的。例如:

In [65]:
import heapq

nums = [1, 8, 2, 23, 7 ,-4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3,nums))

[42, 37, 23]


In [66]:
print(heapq.nsmallest(3,nums))

[-4, 1, 2]


这两个函数都可以接收一个参数key，从而允许它们工作在更加复杂的数据结构之上。例如:

In [70]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price':91.1},
    {'name': 'AAPL', 'shares': 50, 'price':543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares':75, 'price': 115.65}
    ]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
cheap

[{'name': 'YHOO', 'price': 16.35, 'shares': 45},
 {'name': 'FB', 'price': 21.09, 'shares': 200},
 {'name': 'HPQ', 'price': 31.75, 'shares': 35}]

In [71]:
expensive

[{'name': 'AAPL', 'price': 543.22, 'shares': 50},
 {'name': 'ACME', 'price': 115.65, 'shares': 75},
 {'name': 'IBM', 'price': 91.1, 'shares': 100}]

#### 4-3 讨论
如果正在寻找最大或最小的N个元素，且同集合中元素的总数目项目，N很小，纳闷下面这些函数可以提供更好的性能。这些函数首先会在底层将数据转化成列表，且元素会以堆的顺序排列。例如:

In [72]:
nums = [1,8,2,23,7,-4,18,23,42,37,2]
import heapq
heap = list(nums)
heapq.heapify(heap)
heap

[-4, 2, 1, 23, 7, 2, 18, 23, 42, 37, 8]

堆最重要的特性就是heap[0]总是最小的那个元素。此外，接下来的元素可依次通过heapq.heappop()方法轻松找到。该方法会将第一个元素弹出，然后以第二小的元素取代之。例如要找到第3小的元素，可以这样做:

In [73]:
heapq.heappop(heap)

-4

In [74]:
heapq.heappop(heap)

1

In [75]:
heapq.heappop(heap)

2

当所要找的元素数量相对较少，函数nlargest()和nsamllest()才是最合适的。如果只是简单地想要找到最小或最大的元素(N=1)，那么用min()和max()会更加快。同样，如果N和集合本身的大小差不多，通常更快的方法是对集合排序，然后做切片操作。

### 5 实现优先级队列
#### 5-1 问题
我们想要实现一个队列，ta能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。
#### 5-2 解决方案
下面的类利用heapq模块实现了一个简单的优先级队列:

In [80]:
import heapq
class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    def pop(self):
        return heapq.heappop(self._queue)[-1]

下面是如何使用这个类的例子:

In [81]:
class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)
    
q = PriorityQueue()
q.push(Item('foo'),1)
q.push(Item('bar'),5)
q.push(Item('spam'),4)
q.push(Item('grok'),1)
q.pop()

Item('bar')

In [82]:
q.pop()

Item('spam')

In [83]:
q.pop()

Item('foo')

In [84]:
q.pop()

Item('grok')

我们观察到第一次执行pop()操作时返回的元素具有最高的优先级。我们也观察到拥有相同优先级的两个元素（foo和grok）返回的顺序同它们插入到队列的顺序相同。

#### 5-3 讨论
上面的代码片段核心在于heapq模块的使用。函数heapq.heappush()以及heapq.heappop()分别实现将元素从列表_queue中插入和移除，且保证列表中的第一个元素的优先级最低。在这段代码中，队列以元组(-priority,index,item)的形式组成。把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排列。变量index的作用是为了将具有相同优先级的元素以适当的顺序来排列。
为了说明Item实例是没法进行次序比较的，我们来看下面这个例子:

In [85]:
a = Item('foo')
b = Item('bar')
a<b

TypeError: unorderable types: Item() < Item()

如果以元组(priority,item)的形式来表示元素，那么只要优先级不同，它们就可以进行比较。但是，如果两个元组的优先级相同，作比较操作还是会失败。例如:

In [86]:
a = (1, Item('foo'))
b = (5, Item('bar'))
a<b

True

In [87]:
c = (1, Item('grok'))
a<c

TypeError: unorderable types: Item() < Item()

通过引入额外的索引值，以(prioroty,index,item)的方式建立元组，就可以完全避免这个问题。因为没有哪两个元组会有相同的索引值。

In [ ]:
a = 